In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine

# API-Anfrage durchführen
url = "https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2022-06-17/2023-06-17"
params = {
    "adjusted": "true",
    "sort": "asc",
    "limit": 120,
    "apiKey": "AtE7dRJG_TP0fPRBCnu1gGpNMe580JCY"
}
response = requests.get(url, params=params)
data = response.json()

# Datenverarbeitung
results = data['results']
df = pd.DataFrame(results)
df['timestamp'] = pd.to_datetime(df['t'], unit='ms')

In [3]:
import pyodbc
import requests
import pandas as pd

# Verbindungsstring
conn_str = (
    r'Driver=SQL Server;'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur Datenbank herstellen
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

# API-Anfrage durchführen
url = "https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2022-06-17/2023-06-17"
params = {
    "adjusted": "true",
    "sort": "asc",
    "limit": 500,
    "apiKey": "AtE7dRJG_TP0fPRBCnu1gGpNMe580JCY"
}
response = requests.get(url, params=params)
data = response.json()

# Datenverarbeitung
results = data['results']
df = pd.DataFrame(results)
df['timestamp'] = pd.to_datetime(df['t'], unit='ms')

# Tabelle erstellen
table_name = "stock_data"  # Name der Zieltabelle
create_table_query = f"CREATE TABLE {table_name} (timestamp datetime, o decimal(18, 8), h decimal(18, 8), l decimal(18, 8), c decimal(18, 8), v decimal(18, 8))"
cursor.execute(create_table_query)

# Daten in die Tabelle einfügen
for _, row in df.iterrows():
    insert_query = f"INSERT INTO {table_name} (timestamp, o, h, l, c, v) VALUES (?, ?, ?, ?, ?, ?)"
    cursor.execute(insert_query, row['timestamp'], row['o'], row['h'], row['l'], row['c'], row['v'])

# Änderungen in der Datenbank übernehmen
cnxn.commit()

print("Daten wurden erfolgreich in die SQL-Tabelle eingefügt.")


ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC SQL Server Driver][SQL Server]There is already an object named 'stock_data' in the database. (2714) (SQLExecDirectW)")

In [ ]:
resultsarray
c*number

The close price for the symbol in the given time period.
h*number

The highest price for the symbol in the given time period.
l*number

The lowest price for the symbol in the given time period.
ninteger

The number of transactions in the aggregate window.
o*number

The open price for the symbol in the given time period.
otcboolean

Whether or not this aggregate is for an OTC ticker. This field will be left off if false.
t*integer

The Unix Msec timestamp for the start of the aggregate window.
v*number

The trading volume of the symbol in the given time period.
vwnumber

The volume weighted average price.


In [5]:
import pyodbc
import requests
import pandas as pd

# Verbindungsstring
conn_str = (
    r'Driver=SQL Server;'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur Datenbank herstellen
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

# Tabelle prüfen
table_name = "stock_data"  # Name der Zieltabelle
start_date = "2022-06-17"  # Startdatum, ab dem Daten überprüft werden sollen
check_data_query = f"SELECT MAX(timestamp) FROM {table_name}"
cursor.execute(check_data_query)
row = cursor.fetchone()
max_timestamp = row[0]

# Daten nur einfügen, wenn keine vorhanden sind
if max_timestamp is None or max_timestamp.date() < pd.to_datetime(start_date).date():
    # API-Anfrage durchführen
    url = "https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2022-06-17/2023-06-17"
    params = {
        "adjusted": "true",
        "sort": "asc",
        "limit": 500,
        "apiKey": "AtE7dRJG_TP0fPRBCnu1gGpNMe580JCY"
    }
    response = requests.get(url, params=params)
    data = response.json()

    # Datenverarbeitung
    results = data['results']
    df = pd.DataFrame(results)
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')

    # Tabelle erstellen
    create_table_query = f"CREATE TABLE {table_name} (timestamp datetime, o decimal(18, 8), h decimal(18, 8), l decimal(18, 8), c decimal(18, 8), v decimal(18, 8))"
    cursor.execute(create_table_query)

    # Daten in die Tabelle einfügen
    for _, row in df.iterrows():
        insert_query = f"INSERT INTO {table_name} (timestamp, o, h, l, c, v) VALUES (?, ?, ?, ?, ?, ?)"
        cursor.execute(insert_query, row['timestamp'], row['o'], row['h'], row['l'], row['c'], row['v'])

    # Änderungen in der Datenbank übernehmen
    cnxn.commit()

    print("Daten wurden erfolgreich in die SQL-Tabelle eingefügt.")
else:
    print("Die Tabelle enthält bereits Daten ab dem angegebenen Startdatum. Es wurden keine weiteren Daten hinzugefügt.")


Die Tabelle enthält bereits Daten ab dem angegebenen Startdatum. Es wurden keine weiteren Daten hinzugefügt.


In [6]:
import pyodbc
import requests
import pandas as pd

# Verbindungsstring
conn_str = (
    r'Driver=SQL Server;'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur Datenbank herstellen
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

# Tabelle prüfen
table_name = "stock_data"  # Name der Zieltabelle
start_date = "2022-06-17"  # Startdatum, ab dem Daten überprüft werden sollen

# Daten aus der Tabelle in einen DataFrame laden
select_query = f"SELECT * FROM {table_name}"
df_existing_data = pd.read_sql(select_query, cnxn)

# API-Anfrage durchführen
url = "https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2022-06-17/2023-06-17"
params = {
    "adjusted": "true",
    "sort": "asc",
    "limit": 500,
    "apiKey": "AtE7dRJG_TP0fPRBCnu1gGpNMe580JCY"
}
response = requests.get(url, params=params)
data = response.json()

# Datenverarbeitung
results = data['results']
df_new_data = pd.DataFrame(results)
df_new_data['timestamp'] = pd.to_datetime(df_new_data['t'], unit='ms')

# Daten nur einfügen, wenn sie noch nicht vorhanden sind
for _, row in df_new_data.iterrows():
    timestamp = row['timestamp']
    existing_row = df_existing_data[df_existing_data['timestamp'] == timestamp]
    if existing_row.empty:
        # Daten in die Tabelle einfügen
        insert_query = f"INSERT INTO {table_name} (timestamp, o, h, l, c, v) VALUES (?, ?, ?, ?, ?, ?)"
        cursor.execute(insert_query, timestamp, row['o'], row['h'], row['l'], row['c'], row['v'])

# Änderungen in der Datenbank übernehmen
cnxn.commit()

print("Daten wurden erfolgreich in die SQL-Tabelle eingefügt.")


Daten wurden erfolgreich in die SQL-Tabelle eingefügt.
